# 02 - Data Cleaning

Objective:
Prepare raw logs for analysis by correcting inconsistencies and handling data quality issues.

Planned Tasks:
- Convert timestamp column to datetime
- Remove duplicates
- Handle missing values
- Validate data ranges
- Standardize categorical values